In [1]:
from pprint import pprint

In [2]:
import os
import mbison.client.core as dmda

auth = dmda.DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)
auth

DomoAuth(domo_instance='domo-community', username=None)

In [3]:
import mbison.feature.cards as dmac
import mbison.feature.appdb as dmdb


def get_source_code_collection_for_ddx(card_id, auth, debug_api : bool = False):
    """
    identifies the collection that stores the source code for a ddx brick

    note card metadata includes "domoapps" which identifies the datastore that feeds an app.
    the one or many collections can be associated with a datastore / app
    for ddx, the source code is stored in the collection 'ddx_app_client_code'
    """

    res = dmac.get_card_by_id(card_id = card_id, auth = auth, debug_api= debug_api)
    
    datastore_id = res.response['domoapp']['id']

    res = dmdb.get_collections(auth=auth, debug_api= debug_api,
                    datastore_id= datastore_id
                    )
    
    source_code_collection = next((obj for obj in res.response if obj['name'] == 'ddx_app_client_code' ),None)

    if not source_code_collection:
        raise Exception(f'unable to to retrieve source code collection for {card_id}')
    
    
    return source_code_collection

test_card_id = '421373262'
test_collection = get_source_code_collection_for_ddx(card_id = test_card_id, auth = auth, debug_api = False)
test_collection

{'id': '9699b866-056d-43ca-9c88-ce32f899d0e7',
 'datastoreId': 'ad37089f-6772-4710-bcd0-aba812c0fe14',
 'defaultPermissions': 'read,read_content',
 'requiredAuthorities': {'CREATE': ['domoapps.edit'],
  'CREATE_CONTENT': ['domoapps.edit'],
  'DELETE': ['domoapps.edit'],
  'DELETE_CONTENT': ['domoapps.edit'],
  'UPDATE': ['domoapps.edit'],
  'UPDATE_CONTENT': ['domoapps.edit']},
 'owner': 1950208331,
 'name': 'ddx_app_client_code',
 'datasourceId': None,
 'schema': None,
 'filters': None,
 'syncEnabled': False,
 'syncRequired': False,
 'fullReplaceRequired': False,
 'lastSync': None,
 'createdOn': '2024-07-04T17:13:56.572Z',
 'updatedOn': '2024-07-04T17:13:56.572Z',
 'updatedBy': 1950208331}

In [4]:
def get_source_code_from_collection(collection_id, auth, debug_api : bool = False):
    """
    extracts the document in a collection containing the ddx source code
    will autmoatically handle collection_sharing if collection is not currently shared with authenticated user
    
    source code will always be stored in document.content.htmlBlank
    """
    
    source_code_document = None
    attempt = 0

    while not source_code_document and attempt <= 1:
        try:
            res = dmdb.query_collection_documents(auth=auth,
                                        debug_api= debug_api,
                                        collection_id = collection_id,query = {}
                                        )
    
            source_code_document = next((obj for obj in res.response 
                                        #  if obj['content'].get('htmlBlank')), 
                                        ), None)
            
            # print(source_code_document['content'].keys())

        except dmdb.AppDb_API_Exception: 
            user_id = auth.who_am_i()['id']
        
            dmdb.modify_collection_permissions(
                collection_id = collection_id,
                user_id=user_id,
                permission= dmdb.Collection_Permission_Enum.READ_CONTENT,
                auth = auth
            )

        attempt +=1

    # if not source_code_document:
    #     raise Exception(f"document containing 'htmlBlank' not retrieved from collection - {collection_id}")

    return source_code_document

    
get_source_code_from_collection(collection_id=test_collection['id'], auth = auth)

{'id': '038104d7-a988-43bc-a8ae-3630805fa149',
 'datastoreId': 'ad37089f-6772-4710-bcd0-aba812c0fe14',
 'collectionId': '9699b866-056d-43ca-9c88-ce32f899d0e7',
 'syncRequired': True,
 'owner': '1950208331',
 'createdBy': '1950208331',
 'createdOn': '2024-07-04T17:14:02.346Z',
 'updatedOn': '2024-07-29T13:47:36.151Z',
 'updatedBy': '1950208331',
 'content': {'htmlBlank': {'js': '// DDX Bricks Wiki - See https://developer.domo.com/docs/ddx-bricks/getting-started-using-ddx-bricks\n// for tips on getting started, linking to Domo data and debugging your app\n \n//Available globals\nvar domo = window.domo; // For more on domo.js: https://developer.domo.com/docs/dev-studio-guides/domo-js#domo.get\nvar datasets = window.datasets;\n\n//Step 1. Select your dataset(s) . dataset0 will be the default alias but you can choose other datasets and alias via the dataset dropdown and select dataset options.\n//Current dataset alias is dataset0 and selected dataset is "EXAMPLE SALES DATA"\n\n\n//Step 2. Q

In [5]:
from typing import Callable
import pandas as pd

def process_card(card_id, auth, debug_api : bool = False, process_card_source_code_document_fn:  Callable = None):
    collection = get_source_code_collection_for_ddx(card_id = card_id, auth = auth, debug_api = debug_api)

    source_code_document = get_source_code_from_collection(collection_id= collection['id'], auth = auth, debug_api = debug_api)

    if process_card_source_code_document_fn:
        return process_card_source_code_document_fn(source_code_document)
    
    return source_code_document


def process_card_source_code_document_fn(doc) :
    return {
        'created_on' : doc['createdOn'],
        'updated_on' : doc['updatedOn'],
        'ddx_type' : next(iter(doc['content'])),
        **doc['content'][next(iter(doc['content']))]
                          
    }

process_card(card_id=test_card_id, auth = auth, process_card_source_code_document_fn=process_card_source_code_document_fn)

{'created_on': '2024-07-04T17:14:02.346Z',
 'updated_on': '2024-07-29T13:47:36.151Z',
 'ddx_type': 'htmlBlank',
 'js': '// DDX Bricks Wiki - See https://developer.domo.com/docs/ddx-bricks/getting-started-using-ddx-bricks\n// for tips on getting started, linking to Domo data and debugging your app\n \n//Available globals\nvar domo = window.domo; // For more on domo.js: https://developer.domo.com/docs/dev-studio-guides/domo-js#domo.get\nvar datasets = window.datasets;\n\n//Step 1. Select your dataset(s) . dataset0 will be the default alias but you can choose other datasets and alias via the dataset dropdown and select dataset options.\n//Current dataset alias is dataset0 and selected dataset is "EXAMPLE SALES DATA"\n\n\n//Step 2. Query your dataset(s): https://developer.domo.com/docs/dev-studio-references/data-api\nvar fields = []; // keeping empty requests all fields\nvar groupby = []; // keeping empty to not use groupby\nvar query = `/data/v1/${datasets[0]}?fields=${fields.join()}&grou

In [6]:
def download_ddx_source_code(
    source_code_document,
    download_folder,
    process_card_source_code_document_fn: Callable,
):
    source_code = process_card_source_code_document_fn(source_code_document)

    dmut.upsert_folder(folder_path=download_folder)

    with open(f"{download_folder}/index.html", "w+") as f:
        f.write(source_code["html"])

    with open(f"{download_folder}/app.js", "w+") as f:
        f.write(source_code["js"])

    with open(f"{download_folder}/styles.css", "w+") as f:
        f.write(source_code["css"])

    return True

In [20]:
import mbison.feature.enterprise_apps as dmea
import mbison.feature.cards as dmca
import mbison.feature.appdb as dmdb
import mbison.client.utils as dmut


def get_app_meta(app_id, auth, debug_api : bool = False):
    endpoint=  f'/domoapps/apps/v2/{app_id}'

    return dmda.domo_api_request(
        endpoint=endpoint, auth=auth, request_type="get", debug_api=debug_api
    )



def process_card(card_id, auth, base_folder: str , debug_api : bool = False, ):
    card_meta = dmca.get_card_by_id(card_id = card_id, auth = auth, debug_api= debug_api).response
        
    app_id = card_meta['domoapp']['id'] 
    
    collections = dmdb.get_collections(auth=auth, debug_api= False,
                      datastore_id = app_id
                      ).response
    
    app_meta = get_app_meta(app_id = app_id , auth = auth).response

    design_id = app_meta['designId']

    design_meta = dmea.get_app_by_id(auth = auth, design_id= design_id).response
    design_version = design_meta['latestVersion']

    ddx_collection = next((collection for collection in collections if collection['name'] == 'ddx_app_client_code'), None)


    source_code = None
    if not ddx_collection:
        source_code = dmea.get_app_source_by_version(
            auth=auth,
            design_id=design_id,
            version=design_version,
            debug_api=False,
            download_path= f"{base_folder}/{design_meta['name']}/{design_version}/app.zip",
        )

    else:
        source_code_document = get_source_code_from_collection(collection_id=ddx_collection['id'], auth = auth)
        
        download_folder = f"{os.path.join(base_folder , str(card_id))}/"

        download_ddx_source_code(source_code_document, download_folder=download_folder, process_card_source_code_document_fn = process_card_source_code_document_fn)

    return {'app_meta': app_meta, 'design_meta': design_meta, 'collections': collections , "source_code" : source_code}


test_card_id = '1405984654' # enterprise app
# test_card_id = '421373262' # ddx

pprint(process_card(test_card_id, auth, base_folder= '../../TEST/', debug_api= False))


done writing stream
{'app_meta': {'accountMapping': [],
              'actionMapping': [],
              'collections': [],
              'context': {'accountMapping': [],
                          'actionMapping': [],
                          'collections': [],
                          'createdDate': '2022-12-02T17:29:22Z',
                          'designId': '8c16c8ab-c068-4110-940b-f738d7146efc',
                          'designVersion': None,
                          'id': '908c9814-4b48-4ba2-8990-03423819de9c',
                          'isDisabled': False,
                          'lastModifiedDate': '2022-12-02T17:29:25Z',
                          'mapping': [],
                          'packageMapping': [],
                          'workflowMapping': []},
              'createdDate': '2022-12-02T17:29:25Z',
              'designId': '8c16c8ab-c068-4110-940b-f738d7146efc',
              'designVersion': None,
              'id': '87784693-7e11-4f4e-9e20-50525b8fc6f6',


In [21]:

def get_all_domoapps(auth :dmda.DomoAuth, temp_limit = None):
    """searches adminsummary API for all cards with cardtype domoapp"""
    
    query = {
            "includeCardTypeClause": True,
            "cardTypes": ["domoapp"],
            "ascending": True,
            "orderBy": "cardTitle",
        }
    
    res = dmac.search_cards(auth = auth, query = query)

    if temp_limit:
        res.response = res.response[:temp_limit]
    
    cards = [ (dmac.get_card_by_id(obj['id'], auth = auth)).response for obj in res.response]
    
    print(f'{len(cards)} ddx bricks retrieverd from {auth.domo_instance}')
    
    return cards


cards = get_all_domoapps(auth = auth, temp_limit= 10)[0:2]

10 ddx bricks retrieverd from domo-community


In [22]:
def process_instance(
    auth: dmda.DomoAuth,
    base_folder : str,
    temp_limit=20,
):
    cards = get_all_domoapps(auth=auth, temp_limit=temp_limit)
    base_folder = f"{base_folder}/{auth.domo_instance}/"

    print(base_folder)
    
    return [
        process_card(
            card_id=card["id"],
            auth=auth,
            base_folder = base_folder
        )
        for card in cards
    ]


process_instance(
    auth=auth,
    temp_limit=10,
    base_folder = '../../TEST'
)

10 ddx bricks retrieverd from domo-community
../../TEST/domo-community/
done writing stream
done writing stream


[{'app_meta': {'id': 'aace1266-dc7f-42cb-8595-46069d91a703',
   'context': {'id': '708cff46-2a20-4ddd-b6dc-eb7a457619c5',
    'designId': '897f9ffc-1ce2-4247-94d3-7afcb0192abb',
    'designVersion': '1.6.1',
    'mapping': [{'alias': 'dataset0',
      'dataSetId': '2b0016a7-02bd-45b4-a9fe-b33fbf5b365e',
      'fields': [],
      'dql': None},
     {'alias': 'dataset1',
      'dataSetId': 'b995e5ba-b1cc-4dff-89f7-d8949c0d0aad',
      'fields': [],
      'dql': None},
     {'alias': 'dataset2',
      'dataSetId': 'b995e5ba-b1cc-4dff-89f7-d8949c0d0aad',
      'fields': [],
      'dql': None}],
    'collections': [],
    'accountMapping': [],
    'actionMapping': [],
    'workflowMapping': [],
    'packageMapping': [],
    'createdDate': '2024-03-29T01:33:40Z',
    'lastModifiedDate': '2024-03-29T01:51:12Z',
    'isDisabled': False},
   'createdDate': '2024-03-28T23:25:14Z',
   'designId': '897f9ffc-1ce2-4247-94d3-7afcb0192abb',
   'designVersion': '1.6.1',
   'mapping': [{'alias': 'datase